<a href="https://colab.research.google.com/github/mmetawei/AFQC/blob/main/DL_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print(" TRAINING DEEP NEURAL NETWORK")
print("="*70)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.optimizers import Adam

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Build DNN model
def build_dnn_model(input_dim):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),

        # First hidden layer
        layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        # Second hidden layer
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        # Third hidden layer
        layers.Dense(32, activation='relu'),

        # Output layer (regression)
        layers.Dense(1, activation='linear')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae', 'mse']
    )

    return model

# Create model
dnn_model = build_dnn_model(X_train_scaled.shape[1])
print("\n Model Architecture:")
dnn_model.summary()

# Callbacks
callbacks_list = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-6,
        verbose=1
    )
]

# Train the model
print("\n Training DNN...")
history = dnn_model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=callbacks_list,
    verbose=1
)

# Evaluate
print("\n Evaluating DNN...")
y_pred_dnn = dnn_model.predict(X_test_scaled).flatten()

dnn_r2 = r2_score(y_test, y_pred_dnn)
dnn_rmse = np.sqrt(mean_squared_error(y_test, y_pred_dnn))
dnn_mae = mean_absolute_error(y_test, y_pred_dnn)

print(f" DNN Performance:")
print(f"   R² Score: {dnn_r2:.6f}")
print(f"   RMSE: {dnn_rmse:.6f}")
print(f"   MAE: {dnn_mae:.6f}")

# Add DNN to comparison
dnn_result = {
    'Model': 'Deep Neural Network',
    'R²': dnn_r2,
    'RMSE': dnn_rmse,
    'MAE': dnn_mae,
    'MSE': mean_squared_error(y_test, y_pred_dnn),
    'Train Time (s)': np.nan  # Not tracking for DNN
}

# Update results
results_df = pd.concat([results_df, pd.DataFrame([dnn_result])], ignore_index=True)
results_df = results_df.sort_values('R²', ascending=False).reset_index(drop=True)

print("\n" + "="*70)
print(" UPDATED MODEL RANKING (with DNN)")
print("="*70)

# Show updated ranking
display(results_df.style.background_gradient(subset=['R²', 'RMSE'], cmap='RdYlGn'))

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].set_title('DNN Training History - Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Training MAE')
axes[1].plot(history.history['val_mae'], label='Validation MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('DNN Training History - MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare DNN with best traditional model
best_traditional_r2 = results_df[results_df['Model'] != 'Deep Neural Network'].iloc[0]['R²']
dnn_position = results_df[results_df['Model'] == 'Deep Neural Network'].index[0] + 1

print(f"\n DNN COMPARISON:")
print(f"   DNN Rank: #{dnn_position} out of {len(results_df)} models")
print(f"   Best traditional model: {best_traditional_r2:.6f}")
print(f"   DNN performance: {dnn_r2:.6f}")
print(f"   Difference: {dnn_r2 - best_traditional_r2:.6f}")

# Save DNN model
dnn_model.save('dnn_model.h5')
print(f"\n DNN model saved as: 'dnn_model.h5'")
